In [ ]:
#FINAL PROJECT RECOMMENDATION SYSTEM 

In [82]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from difflib import get_close_matches
import numpy as np


In [40]:
moviedata=pd.read_csv(r"C:\Users\Lenovo\Desktop\intership\DSML 10 Am-20240804T125825Z-001\DSML 10 Am\Project\Project Dataset\movies.csv")
moviedata

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
10324,146684,Cosmic Scrat-tastrophe (2015),Animation|Children|Comedy
10325,146878,Le Grand Restaurant (1966),Comedy
10326,148238,A Very Murray Christmas (2015),Comedy
10327,148626,The Big Short (2015),Drama


In [42]:
ratingdata=pd.read_csv(r"C:\Users\Lenovo\Desktop\intership\DSML 10 Am-20240804T125825Z-001\DSML 10 Am\Project\Project Dataset\ratings.csv")
ratingdata

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523
...,...,...,...,...
105334,668,142488,4.0,1451535844
105335,668,142507,3.5,1451535889
105336,668,143385,4.0,1446388585
105337,668,144976,2.5,1448656898


In [44]:
moviedata.nunique()

movieId    10329
title      10327
genres       938
dtype: int64

In [9]:
moviedata.shape

(10329, 3)

In [46]:
mergeddata=pd.merge(moviedata,ratingdata,on="movieId")
mergeddata.head(10)

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,5.0,859046895
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1303501039
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,5.0,858610933
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.0,850815810
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14,4.0,851766286
5,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,5.0,1350206819
6,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,28,3.0,884098350
7,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,29,4.0,846942580
8,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,30,4.5,1292690069
9,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,31,4.0,832397496


In [48]:
mergeddata.groupby('title')['rating'].mean().sort_values(ascending=False).head() 

title
Saddest Music in the World, The (2003)    5.0
Interstate 60 (2002)                      5.0
Gunfighter, The (1950)                    5.0
Heima (2007)                              5.0
Limelight (1952)                          5.0
Name: rating, dtype: float64

In [12]:
mergeddata.shape

(105339, 6)

In [50]:
ratings_count = mergeddata.groupby('title')['rating'].count().reset_index()
ratings_count.columns = ['title', 'rating_count'] 
mergeddata = pd.merge(mergeddata, ratings_count, on='title', how='inner')
mergeddata = mergeddata.sort_values(by='rating_count', ascending=True)
mergeddata


,movieId,title,genres,userId,rating,timestamp,rating_count
105338,149532,Marco Polo: One Hundred Eyes (2015),(no genres listed),475,4.0,1451223429,1
87436,26347,"Irony of Fate, or Enjoy Your Bath! (Ironiya su...",Comedy|Drama|Romance,405,5.0,1296442373,1
101602,88599,Come Undone (Cosa voglio di più) (2010),Drama,575,3.0,1312478707,1
101601,88595,"Temptress, The (1926)",Drama|Romance,15,5.0,1447478795,1
101598,88566,Mysteries of Lisbon (Mistérios de Lisboa) (2010),Drama|Mystery,668,3.0,1334982541,1
...,...,...,...,...,...,...,...
8834,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,253,4.0,1353228031,325
8833,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,250,4.5,1107160162,325
8832,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,246,5.0,845315848,325
8830,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,244,3.0,953168435,325


In [52]:
#QUESTION 1
def recommend_movies(mergeddata, genre, min_reviews, no_recommendation):

    filtered_df = mergeddata[(mergeddata['genres'] == genre) & (mergeddata['rating_count'] >= min_reviews)]
    sorted_df = filtered_df.sort_values(by='rating', ascending=False)
    return sorted_df.head(no_recommendation)

genre = 'Drama'
min_reviews = 100
no_recommendation = 5

recommendations = recommend_movies(mergeddata, genre,min_reviews, no_recommendation)
print(recommendations)




       movieId                                   title genres  userId  rating  \
28726     1193  One Flew Over the Cuckoo's Nest (1975)  Drama     550     5.0   
28753     1193  One Flew Over the Cuckoo's Nest (1975)  Drama     643     5.0   
44421     1961                         Rain Man (1988)  Drama     432     5.0   
28633     1193  One Flew Over the Cuckoo's Nest (1975)  Drama      98     5.0   
28733     1193  One Flew Over the Cuckoo's Nest (1975)  Drama     568     5.0   

        timestamp  rating_count  
28726  1425578716           143  
28753  1300442803           143  
44421   957671912           113  
28633   955174718           143  
28733  1142789368           143  


In [54]:
mergeddata.head()

,movieId,title,genres,userId,rating,timestamp,rating_count
105338,149532,Marco Polo: One Hundred Eyes (2015),(no genres listed),475,4.0,1451223429,1
87436,26347,"Irony of Fate, or Enjoy Your Bath! (Ironiya su...",Comedy|Drama|Romance,405,5.0,1296442373,1
101602,88599,Come Undone (Cosa voglio di più) (2010),Drama,575,3.0,1312478707,1
101601,88595,"Temptress, The (1926)",Drama|Romance,15,5.0,1447478795,1
101598,88566,Mysteries of Lisbon (Mistérios de Lisboa) (2010),Drama|Mystery,668,3.0,1334982541,1


In [56]:
#QUESTION 2

datav = pd.DataFrame(mergeddata)

count_vectorizer = CountVectorizer(tokenizer=lambda x: x.split('|'))
genre_matrix = count_vectorizer.fit_transform(datav['genres'])

def get_recommendations(datav, title, num_recommendations):
 
    movie_titles = datav['title'].tolist()
    closest_matches = get_close_matches(title, movie_titles, n=1, cutoff=0.1)
    if not closest_matches:
        return "No matching movie found."
    
    best_match = closest_matches[0]
    
    
    indices = pd.Series(datav.index, index=datav['title']).drop_duplicates()
    idx = indices[best_match]
    
    chunk_size = 1000
    sim_scores = []
    
    for start in range(0, genre_matrix.shape[0], chunk_size):
        end = min(start + chunk_size, genre_matrix.shape[0])
        chunk_cosine_sim = cosine_similarity(genre_matrix[start:end], genre_matrix[idx])
        sim_scores.extend(chunk_cosine_sim)
    
    sim_scores = [(i, score[0]) for i, score in enumerate(sim_scores)]
       
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:num_recommendations + 1]
 
    movie_indices = [i[0] for i in sim_scores]
    
    return datav['title'].iloc[movie_indices]


title = 'Toy Story'
num_recommendations = 3

recommended_movies = get_recommendations(datav, title, num_recommendations)
print(recommended_movies)


D:\Users\Lenovo\Py1\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


59593                      Rawhead Rex (1986)
87221                 Isle of the Dead (1945)
87312    Hush... Hush, Sweet Charlotte (1964)
Name: title, dtype: object


In [80]:
#QUESTION 3
#loading the above table columns into the code
user_item_matrix = datav.pivot_table(index='userId', columns='movieId', values='rating_count').fillna(0)


user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

def get_recommendations(userId, movieId, num_recommendations):
    
    similar_users = user_siilarity_df[userId].sort_values(ascending=False).index[1:movieId+1]

   
    similar_users_ratings = user_item_matrix.loc[similar_users].mean(axis=0)
    similar_users_ratings = similar_users_ratings.sort_values(ascending=False)
    
    
    movies_already_rated = user_item_matrix.loc[userId][user_item_matrix.loc[userId] > 0].index
    recommendations = similar_users_ratings.drop(movies_already_rated).head(num_recommendations)
    
    
    recommended_movies = datav[datav['movieId'].isin(recommendations.index)]['title'].unique()
    
    
    result = pd.DataFrame({'S.No': range(1, len(recommended_movies) + 1), 'Movie Title': recommended_movies})
    
    return result

userId = 1
movieId = 100
num_recommendations = 5


recommended_movies = get_recommendations(userId, movieId, num_recommendations)
print(recommended_movies)


   S.No                                Movie Title
0     1                     Terminator, The (1984)
1     2                            Die Hard (1988)
2     3  Indiana Jones and the Last Crusade (1989)
3     4                 Princess Bride, The (1987)
4     5                           Toy Story (1995)
